In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from scripts.kaggle.helpers import make_submission
import datetime
from tqdm import tqdm
import gc
import itertools
from multiprocessing import Pool
import pickle
import collections
import math

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# %pylab inline
# pylab.rcParams['figure.figsize'] = (12, 16)
plt.figure(figsize=(12,16))

%load_ext autotime

In [3]:
HEADER = ["fecha_dato", "ncodpers", "ind_empleado",
          "pais_residencia", "sexo", "age", "fecha_alta",
          "ind_nuevo", "antiguedad", "indrel", "ult_fec_cli_1t",
          "indrel_1mes", "tiprel_1mes", "indresi", "indext",
          "conyuemp", "canal_entrada", "indfall", "tipodom",
          "cod_prov", "nomprov", "ind_actividad_cliente",
          "renta", "segmento", "ind_ahor_fin_ult1",
          "ind_aval_fin_ult1", "ind_cco_fin_ult1",
          "ind_cder_fin_ult1", "ind_cno_fin_ult1",
          "ind_ctju_fin_ult1", "ind_ctma_fin_ult1",
          "ind_ctop_fin_ult1", "ind_ctpp_fin_ult1",
          "ind_deco_fin_ult1", "ind_deme_fin_ult1",
          "ind_dela_fin_ult1", "ind_ecue_fin_ult1",
          "ind_fond_fin_ult1", "ind_hip_fin_ult1",
          "ind_plan_fin_ult1", "ind_pres_fin_ult1",
          "ind_reca_fin_ult1", "ind_tjcr_fin_ult1",
          "ind_valo_fin_ult1", "ind_viv_fin_ult1",
          "ind_nomina_ult1", "ind_nom_pens_ult1",
          "ind_recibo_ult1"]

time: 10.9 ms


In [37]:
def get_persons_of_feature(train):
    persons_of_feature = dict()
    for feat in HEADER[24:]:
        persons_of_feature[feat] = set(train.loc[train[feat] == 1]["ncodpers"])
    return persons_of_feature
# for feat in header[24:]:
#     print feat, len(persons_of_feature[feat])

time: 2.96 ms


In [36]:
def jacobian_similarity(persons_of_feature, featur1, featur2):
    feat1 = persons_of_feature[featur1]
    feat2 = persons_of_feature[featur2]
    len1 = len(feat1)
    len2 = len(feat2)
    intersection = len(feat1.intersection(feat2))
    union = len1 + len2 - intersection
    if len1 == 0 or len1 == 0:
        return 0.0
    return intersection * 1.0 / union

def cosine_similarity(persons_of_feature, featur1, featur2):
    feat1 = persons_of_feature[featur1]
    feat2 = persons_of_feature[featur2]
    len1 = len(feat1)
    len2 = len(feat2)
    intersection = len(feat1.intersection(feat2))
    # union = len1 + len2 - intersection
    if len1 == 0 or len2 == 0:
        return 0.0
    return intersection * 1.0 / math.sqrt(len1*len2)

time: 10.2 ms


In [24]:
def get_similarity_matrices(train_df):
    persons_of_feature = get_persons_of_feature(train_df)
    jacob_matrix = collections.defaultdict(dict)
    cosine_matrix = collections.defaultdict(dict)
    for i in range(24):
        jacob_matrix[i][i] = 1.0
        cosine_matrix[i][i] = 1.0
        for j in range(i + 1, 24):
            temp = jacobian_similarity(persons_of_feature, HEADER[i + 24], HEADER[j+24])
            jacob_matrix[i][j] = temp
            jacob_matrix[j][i] = temp

            temp1 = cosine_similarity(persons_of_feature, HEADER[i+24], HEADER[j+24])
            cosine_matrix[i][j] = temp1
            cosine_matrix[j][i] = temp1
    cosine_pd = pd.DataFrame(cosine_matrix)
    jacob_pd = pd.DataFrame(jacob_matrix)
    return jacob_pd, cosine_pd


def get_persons_of_feature_from_added(added):
    persons_of_feature = dict()
    for feat in HEADER[24:]:
        persons_of_feature[feat] = set(added.loc[added['added_product'] == feat]["ncodpers"])
    return persons_of_feature
# for feat in header[24:]:
#     print feat, len(persons_of_feature[feat])

def get_similarity_matrices_from_added(added_df):
    persons_of_feature = get_persons_of_feature_from_added(added_df)
    jacob_matrix = collections.defaultdict(dict)
    cosine_matrix = collections.defaultdict(dict)
    for i in range(24):
        jacob_matrix[i][i] = 1.0
        cosine_matrix[i][i] = 1.0
        for j in range(i + 1, 24):
            temp = jacobian_similarity(persons_of_feature, HEADER[i + 24], HEADER[j+24])
            jacob_matrix[i][j] = temp
            jacob_matrix[j][i] = temp

            temp1 = cosine_similarity(persons_of_feature, HEADER[i+24], HEADER[j+24])
            cosine_matrix[i][j] = temp1
            cosine_matrix[j][i] = temp1
    cosine_pd = pd.DataFrame(cosine_matrix)
    jacob_pd = pd.DataFrame(jacob_matrix)
    return jacob_pd, cosine_pd

time: 77.9 ms


In [38]:
def get_wts_from_mat(df, row):
    brow = map(bool, row)
    wts = df.loc[brow, :].apply(sum)
    wts[brow] = 0
    return wts

time: 2.52 ms


In [39]:
def get_similarity_features(file_names): #train_file, added_file):
    train_file = file_names[0]
    added_file = file_names[1]
    print added_file
    train = pd.read_csv(train_file, header=None, names=HEADER[1:2] + HEADER[24:], usecols=[1]+range(24,48))
    train.fillna(0, inplace=True)
    for col in HEADER[24:]:
        train[col] = train[col].astype(int)
    if added_file == 'data/test_ver2.csv':
        added_products = pd.read_csv(added_file, usecols=[1])
    else:
        added_products = pd.read_csv(added_file, usecols=[0])
    jacob_pd, cosine_pd = get_similarity_matrices(train)
    train.set_index('ncodpers', inplace=True)
    train_final = pd.DataFrame()
    train_final['ncodpers'] = added_products['ncodpers']
    train_final.set_index('ncodpers', inplace=True)
    for col in HEADER[24:]:
        train_final['jacob_' + col] = [0.0]*train_final.shape[0]
        train_final['cosine_' + col] = [0.0]*train_final.shape[0]

    for i in tqdm(train_final.index):
        row = list(train.loc[i, HEADER[24:]])
        jwts = get_wts_from_mat(jacob_pd, row)
        cwts = get_wts_from_mat(cosine_pd, row)
        train_final.loc[i, ['jacob_' + h for h in HEADER[24:]]] = list(jwts)
        train_final.loc[i, ['cosine_' + h for h in HEADER[24:]]] = list(cwts)
    return train_final, file_names[2]

time: 19.4 ms


In [47]:
def get_similarity_features_custom(file_names): #train_file, added_file):
    train_file = file_names[0]
    added_file = file_names[1]
    print added_file
    train = pd.read_csv(train_file, header=None, names=HEADER[1:2] + HEADER[23:], usecols=[1]+range(23,48))
    
    for col in HEADER[24:]:
        train[col].fillna(0, inplace=True)
        train[col] = train[col].astype(int)
    if added_file == 'data/test_ver2.csv':
        added_products = pd.read_csv(added_file, usecols=[1])
    else:
        added_products = pd.read_csv(added_file, usecols=[0])
    
    train.segmento.fillna('02 - PARTICULARES', inplace=True)
    parti = train.loc[train.segmento == '02 - PARTICULARES', :]
    top = train.loc[train.segmento == '01 - TOP', :]
    uni = train.loc[train.segmento == '03 - UNIVERSITARIO', :]
    jacob_pd_p, cosine_pd_p = get_similarity_matrices(parti)
    jacob_pd_t, cosine_pd_t = get_similarity_matrices(top)
    jacob_pd_u, cosine_pd_u = get_similarity_matrices(uni)
    jacobs = {
        '02 - PARTICULARES': (jacob_pd_p, cosine_pd_p),
        '01 - TOP':(jacob_pd_t, cosine_pd_t),
        '03 - UNIVERSITARIO': (jacob_pd_u, cosine_pd_u)
    }
    train.set_index('ncodpers', inplace=True)
    train_final = pd.DataFrame()
    train_final['ncodpers'] = added_products['ncodpers']
    train_final.set_index('ncodpers', inplace=True)
    for col in HEADER[24:]:
        train_final['jacob_custom_' + col] = [0.0]*train_final.shape[0]
        train_final['cosine_custom_' + col] = [0.0]*train_final.shape[0]

    for i in tqdm(train_final.index):
        row = list(train.loc[i, HEADER[24:]])
        segmento = train.loc[i, 'segmento']
        jwts = get_wts_from_mat(jacobs[segmento][0], row)
        cwts = get_wts_from_mat(jacobs[segmento][1], row)
        train_final.loc[i, ['jacob_custom_' + h for h in HEADER[24:]]] = list(jwts)
        train_final.loc[i, ['cosine_custom_' + h for h in HEADER[24:]]] = list(cwts)
    return train_final, file_names[2]

time: 32.5 ms


In [ ]:
feat, ind = get_similarity_features_custom(('data/train_2015_05_28.csv', 'data/added_product_2015_05_28.csv', 1))

data/added_product_2015_05_28.csv


 54%|█████▍    | 22445/41745 [01:44<01:38, 195.04it/s]

In [8]:
# def get_similarity_features_from_added(file_names): #train_file, added_file):
#     train_file = file_names[0]
#     similarity_file = file_names[1]
#     added_file = file_names[2]
#     print added_file
#     train = pd.read_csv(train_file, header=None, names=HEADER[1:2] + HEADER[24:], usecols=[1]+range(24,48))
#     train.fillna(0, inplace=True)
#     similarity_pd = pd.read_csv(similarity_file)
#     for col in HEADER[24:]:
#         train[col] = train[col].astype(int)
#     if added_file == 'data/test_ver2.csv':
#         added_products = pd.read_csv(added_file, usecols=[1])
#     else:
#         added_products = pd.read_csv(added_file, usecols=[0])
#     jacob_pd, cosine_pd = get_similarity_matrices_from_added(similarity_pd)
#     train.set_index('ncodpers', inplace=True)
#     train_final = pd.DataFrame()
#     train_final['ncodpers'] = added_products['ncodpers']
#     train_final.set_index('ncodpers', inplace=True)
#     for col in HEADER[24:]:
#         train_final['jacob_added_' + col] = [0.0]*train_final.shape[0]
#         train_final['cosine_added_' + col] = [0.0]*train_final.shape[0]

#     for i in tqdm(train_final.index):
#         row = list(train.loc[i, HEADER[24:]])
#         jwts = get_wts_from_mat(jacob_pd, row)
#         cwts = get_wts_from_mat(cosine_pd, row)
#         train_final.loc[i, ['jacob_added_' + h for h in HEADER[24:]]] = list(jwts)
#         train_final.loc[i, ['cosine_added_' + h for h in HEADER[24:]]] = list(cwts)
#     return train_final, jacob_pd, cosine_pd, file_names[3]

time: 21 ms


In [8]:
file_names_list = [('data/train_2015_05_28.csv', 'data/added_product_2015_05_28.csv', 1),
                   ('data/train_2015_06_28.csv', 'data/added_product_2015_06_28.csv', 2),
                   ('data/train_2015_07_28.csv', 'data/added_product_2015_07_28.csv', 3),
                   ('data/train_2015_08_28.csv', 'data/added_product_2015_08_28.csv', 4),
                   ('data/train_2015_09_28.csv', 'data/added_product_2015_09_28.csv', 5),
                   ('data/train_2015_10_28.csv', 'data/added_product_2015_10_28.csv', 6),
                   ('data/train_2015_11_28.csv', 'data/added_product_2015_11_28.csv', 7),
                   ('data/train_2015_12_28.csv', 'data/added_product_2015_12_28.csv', 8),
                   ('data/train_2016_01_28.csv', 'data/added_product_2016_01_28.csv', 9),
                   ('data/train_2016_02_28.csv', 'data/added_product_2016_02_28.csv', 10),
                   ('data/train_2016_03_28.csv', 'data/added_product_2016_03_28.csv', 11),
                   ('data/train_2016_04_28.csv', 'data/added_product_2016_04_28.csv', 12)
                  ]

test_file_data = ('data/train_2016_05_28.csv', 'data/test_ver2.csv', 13)

time: 5.86 ms


In [9]:
p = Pool(5)
results = p.map(get_similarity_features, file_names_list)
p.close()
p.join()

data/added_product_2015_07_28.csv
data/added_product_2015_09_28.csv
data/added_product_2015_05_28.csv
data/added_product_2015_06_28.csv
data/added_product_2015_08_28.csv


100%|██████████| 29490/29490 [01:45<00:00, 278.22it/s]


data/added_product_2015_10_28.csv


 85%|████████▌ | 33994/39803 [02:03<00:21, 265.03it/s]

data/added_product_2015_11_28.csv


...

data/added_product_2016_03_28.csv


 69%|██████▊   | 24145/35203 [01:28<00:41, 265.75it/s]

data/added_product_2016_04_28.csv


100%|██████████| 35843/35843 [02:11<00:00, 273.07it/s]


time: 6min 24s


In [10]:
[x[1] for x in results]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

time: 3.21 ms


In [11]:
jacob_train = pd.concat(tuple([x[0] for x in results]))

time: 78.7 ms


In [12]:
jacob_train.shape

(441652, 48)

time: 2.17 ms


In [14]:
pickle.dump(jacob_train, open('data/15.Jacobian_similarity_features.train_data', 'wb'))
make_submission('data/15.Jacobian_similarity_features.train_data', upload_to_s3=True)

Compressing file
Compression done
Uploading file to s3 data/15.Jacobian_similarity_features.train_data.zip
uploading to s3 done
time: 28.7 s


In [15]:
jacob_test, ind = get_similarity_features(test_file_data)

data/test_ver2.csv


100%|██████████| 929615/929615 [1:01:53<00:00, 250.32it/s]

time: 1h 1min 59s


In [16]:
jacob_test.shape

(929615, 48)

time: 2.53 ms


In [18]:
pickle.dump(jacob_test, open('data/15.Jacobian_similarity_features.test_data', 'wb'))
make_submission('data/15.Jacobian_similarity_features.test_data', upload_to_s3=True)

Compressing file
Compression done
Uploading file to s3 data/15.Jacobian_similarity_features.test_data.zip
uploading to s3 done
time: 4min 17s


---
# submission from similarty metrics and an actual submission
---

In [20]:
desc, test_ncodpers, preds, label_encoder = pickle.load(open('data/11.xgb_trained_on_all_months_only_added_users_product_features_generated_from_last_4_months.csv.pickle', 'rb'))

time: 9.52 s


In [21]:
len(label_encoder.classes_)

24

time: 2.72 ms


In [24]:
jacobs = pd.DataFrame()
jacobs['ncodpers'] = jacob_test.index
jacobs.set_index('ncodpers', inplace=True)

time: 71.4 ms


In [25]:
for col in HEADER[24:]:
    jacobs[col] = jacob_test['jacob_'+col]

time: 85.1 ms


In [27]:
preds.shape

(929615, 24)

time: 2.57 ms


In [30]:
jacobs.columns = label_encoder.transform(jacobs.columns)

time: 1.64 ms


In [31]:
jacobs.head()

,0,1,2,3,4,5,6,7,8,9,...,14,17,18,19,21,22,23,16,15,20
ncodpers,,,,,,,,,,,,,,,,,,,,,
15889,0.000884,0.000769,0.0,0.006847,0.415941,0.000005,0.028505,0.275887,0.000000,0.005102,...,0.094846,0.148436,0.012952,0.400850,0.00000,0.000000,0.043409,0.414067,0.434003,0.497496
1170544,0.000082,0.000014,0.0,0.000409,0.011386,0.000005,0.007710,0.110624,0.031354,0.000463,...,0.002969,0.006558,0.001393,0.037283,0.02669,0.026649,0.003207,0.011827,0.012634,0.105950
1170545,0.000082,0.000014,0.0,0.000409,0.011386,0.000005,0.007710,0.110624,0.031354,0.000463,...,0.002969,0.006558,0.001393,0.037283,0.02669,0.026649,0.003207,0.011827,0.012634,0.105950
1170547,0.000082,0.000014,0.0,0.000409,0.011386,0.000005,0.007710,0.110624,0.031354,0.000463,...,0.002969,0.006558,0.001393,0.037283,0.02669,0.026649,0.003207,0.011827,0.012634,0.105950
1170548,0.000082,0.000014,0.0,0.000409,0.011386,0.000005,0.007710,0.110624,0.031354,0.000463,...,0.002969,0.006558,0.001393,0.037283,0.02669,0.026649,0.003207,0.011827,0.012634,0.105950


time: 41 ms


In [32]:
jacobs_1 = jacobs[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]]

time: 168 ms


In [33]:
jacobs_1.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
ncodpers,,,,,,,,,,,,,,,,,,,,,
15889,0.000884,0.000769,0.0,0.006847,0.415941,0.000005,0.028505,0.275887,0.000000,0.005102,...,0.094846,0.434003,0.414067,0.148436,0.012952,0.400850,0.497496,0.00000,0.000000,0.043409
1170544,0.000082,0.000014,0.0,0.000409,0.011386,0.000005,0.007710,0.110624,0.031354,0.000463,...,0.002969,0.012634,0.011827,0.006558,0.001393,0.037283,0.105950,0.02669,0.026649,0.003207
1170545,0.000082,0.000014,0.0,0.000409,0.011386,0.000005,0.007710,0.110624,0.031354,0.000463,...,0.002969,0.012634,0.011827,0.006558,0.001393,0.037283,0.105950,0.02669,0.026649,0.003207
1170547,0.000082,0.000014,0.0,0.000409,0.011386,0.000005,0.007710,0.110624,0.031354,0.000463,...,0.002969,0.012634,0.011827,0.006558,0.001393,0.037283,0.105950,0.02669,0.026649,0.003207
1170548,0.000082,0.000014,0.0,0.000409,0.011386,0.000005,0.007710,0.110624,0.031354,0.000463,...,0.002969,0.012634,0.011827,0.006558,0.001393,0.037283,0.105950,0.02669,0.026649,0.003207


time: 42.7 ms


In [34]:
mat = jacobs_1.as_matrix()

time: 1.28 ms


In [35]:
mat.shape

(929615, 24)

time: 2.92 ms


In [36]:
mat

array([[  8.83835818e-04,   7.68736655e-04,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   4.34094628e-02],
       [  8.19019776e-05,   1.42444309e-05,   0.00000000e+00, ...,
          2.66898708e-02,   2.66487295e-02,   3.20735506e-03],
       [  8.19019776e-05,   1.42444309e-05,   0.00000000e+00, ...,
          2.66898708e-02,   2.66487295e-02,   3.20735506e-03],
       ..., 
       [  4.29703870e-04,   5.40054670e-05,   0.00000000e+00, ...,
          8.61278040e-02,   8.35131729e-02,   1.66351659e-02],
       [  8.19019776e-05,   1.42444309e-05,   0.00000000e+00, ...,
          2.66898708e-02,   2.66487295e-02,   3.20735506e-03],
       [  3.47801892e-04,   3.97610362e-05,   1.10623988e-01, ...,
          5.94379332e-02,   5.68644435e-02,   1.34278108e-02]])

time: 4.2 ms


In [38]:
preds.shape

(929615, 24)

time: 2.51 ms


In [39]:
new_preds = np.multiply(mat, preds)

time: 159 ms


In [65]:
top_t_products = label_encoder.inverse_transform(np.argsort(new_preds, axis=1)
                                                 [:, ::-1][:, :])

time: 1.3 s


In [60]:
submission = pd.DataFrame()
submission['ncodpers'] = test.index
submission['added_products'] = ['ind_recibo_ult1']*submission.shape[0]
submission.set_index('ncodpers', inplace=True)

time: 57 ms


In [42]:
test = pd.read_csv('data/test_ver2.csv')

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


time: 3 s


In [43]:
test.fillna(0, inplace=True)

time: 620 ms


In [44]:
test.set_index('ncodpers', inplace=True)

time: 22.2 ms


In [54]:
test_copy = test.loc[test_ncodpers, :]

time: 1.77 s


In [57]:
test_old = pd.read_csv('data/train_2016_05_28.csv', header=None, names=HEADER)

time: 3.12 s


/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [58]:
test_old.fillna(0, inplace=True)
test_old.set_index('ncodpers', inplace=True)

time: 585 ms


In [55]:
test_copy.head()

,fecha_dato,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,...,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento
ncodpers,,,,,,,,,,,,,,,,,,,,,
869505,2016-06-28,N,ES,H,26,2010-04-13,0,75,1,0,...,N,0,KAT,N,1,4.0,ALMERIA,1,112771.23,02 - PARTICULARES
1477437,2016-06-28,N,ES,H,22,2015-10-13,0,8,1,0,...,N,0,KHQ,N,1,28.0,MADRID,1,NA,03 - UNIVERSITARIO
1533309,2016-06-28,N,ES,H,52,2016-02-19,1,4,1,0,...,N,0,KHN,N,1,12.0,CASTELLON,1,NA,02 - PARTICULARES
1484304,2016-06-28,N,ES,H,25,2015-10-20,0,8,1,0,...,N,0,KHQ,N,1,28.0,MADRID,0,NA,03 - UNIVERSITARIO
1484306,2016-06-28,N,ES,H,22,2015-10-20,0,8,1,0,...,N,0,KHQ,N,1,28.0,MADRID,0,NA,03 - UNIVERSITARIO


time: 51.6 ms


In [66]:
test_copy['xgb_preds'] = [' '.join(x) for x in top_t_products]
test_copy['added_products'] = ['ind_recibo_ult1']*test_copy.shape[0]
for i in tqdm(test_copy.index):
    zipped = zip(HEADER[24:],
                 test_old.loc[i, 'ind_ahor_fin_ult1':'ind_recibo_ult1'])
    products = [x[0] for x in
                [y for y in zipped if y[1] == 1]]
    pred_products = test_copy.loc[i, 'xgb_preds'].split()
    prod_string = ' '.join(filter(lambda x: x not in products, pred_products))
    test_copy.set_value(i, 'added_products', prod_string)

100%|██████████| 929615/929615 [08:45<00:00, 1769.25it/s]

time: 8min 48s


In [61]:
submission.head()

,added_products
ncodpers,
15889,ind_recibo_ult1
1170544,ind_recibo_ult1
1170545,ind_recibo_ult1
1170547,ind_recibo_ult1
1170548,ind_recibo_ult1


time: 5.63 ms


In [67]:
submission.added_products = test_copy.loc[submission.index, 'added_products']

time: 289 ms


In [63]:
submission.head()

,added_products
ncodpers,
15889,ind_recibo_ult1 ind_reca_fin_ult1 ind_dela_fin...
1170544,ind_recibo_ult1 ind_reca_fin_ult1 ind_cno_fin_...
1170545,ind_recibo_ult1 ind_nomina_ult1 ind_nom_pens_u...
1170547,ind_recibo_ult1 ind_reca_fin_ult1 ind_cno_fin_...
1170548,ind_recibo_ult1 ind_reca_fin_ult1 ind_cno_fin_...


time: 5.75 ms


In [68]:
filename = 'data/15.11_submission_weighted_with_jacobian_wts.csv'
description = '15. 11. submission weighted with jacobian weights'
submission.to_csv(filename, columns=['added_products'])
make_submission(filename, description=description, submit=True, compress=True, upload_to_s3=True)

Compressing file
Compression done
Uploading file to s3 data/15.11_submission_weighted_with_jacobian_wts.csv.zip
uploading to s3 done
Uploading submission data/15.11_submission_weighted_with_jacobian_wts.csv.zip
Upload done
time: 1min 25s


---
# get similarity features from added products
---

In [9]:
file_names_list_added = [('data/train_2015_05_28.csv', 'data/added_product_2015_04_28.csv', 'data/added_product_2015_05_28.csv', 1),
                   ('data/train_2015_06_28.csv', 'data/added_product_2015_05_28.csv', 'data/added_product_2015_06_28.csv', 2),
                   ('data/train_2015_07_28.csv', 'data/added_product_2015_06_28.csv', 'data/added_product_2015_07_28.csv', 3),
                   ('data/train_2015_08_28.csv', 'data/added_product_2015_07_28.csv', 'data/added_product_2015_08_28.csv', 4),
                   ('data/train_2015_09_28.csv', 'data/added_product_2015_08_28.csv', 'data/added_product_2015_09_28.csv', 5),
                   ('data/train_2015_10_28.csv', 'data/added_product_2015_09_28.csv', 'data/added_product_2015_10_28.csv', 6),
                   ('data/train_2015_11_28.csv', 'data/added_product_2015_10_28.csv', 'data/added_product_2015_11_28.csv', 7),
                   ('data/train_2015_12_28.csv', 'data/added_product_2015_11_28.csv', 'data/added_product_2015_12_28.csv', 8),
                   ('data/train_2016_01_28.csv', 'data/added_product_2015_12_28.csv', 'data/added_product_2016_01_28.csv', 9),
                   ('data/train_2016_02_28.csv', 'data/added_product_2016_01_28.csv', 'data/added_product_2016_02_28.csv', 10),
                   ('data/train_2016_03_28.csv', 'data/added_product_2016_02_28.csv', 'data/added_product_2016_03_28.csv', 11),
                   ('data/train_2016_04_28.csv', 'data/added_product_2016_03_28.csv', 'data/added_product_2016_04_28.csv', 12)
                  ]

test_file_data_added = ('data/train_2016_05_28.csv', 'data/added_product_2016_04_28.csv', 'data/test_ver2.csv', 13)

time: 6.83 ms


In [14]:
feats_added = get_similarity_features_from_added(file_names_list_added[0])

data/added_product_2015_05_28.csv


100%|██████████| 41745/41745 [02:56<00:00, 235.91it/s]

time: 2min 59s


In [18]:
feats_added[2].head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,0.0,0.0,1.0,0.0,0.0,0.0,0.001468,0.003709,0.000000,0.001362,...,0.007006,0.000000,0.0,0.003404,0.011858,0.007412,0.0,0.005431,0.005814,0.025718
3,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.004490
4,0.0,0.0,0.0,0.0,1.0,0.0,0.008012,0.002699,0.003499,0.004461,...,0.000000,0.010013,0.0,0.003716,0.017259,0.004855,0.0,0.279047,0.285600,0.033797


time: 40.2 ms


In [16]:
feats_added[0].head()

,jacob_added_ind_ahor_fin_ult1,cosine_added_ind_ahor_fin_ult1,jacob_added_ind_aval_fin_ult1,cosine_added_ind_aval_fin_ult1,jacob_added_ind_cco_fin_ult1,cosine_added_ind_cco_fin_ult1,jacob_added_ind_cder_fin_ult1,cosine_added_ind_cder_fin_ult1,jacob_added_ind_cno_fin_ult1,cosine_added_ind_cno_fin_ult1,...,jacob_added_ind_valo_fin_ult1,cosine_added_ind_valo_fin_ult1,jacob_added_ind_viv_fin_ult1,cosine_added_ind_viv_fin_ult1,jacob_added_ind_nomina_ult1,cosine_added_ind_nomina_ult1,jacob_added_ind_nom_pens_ult1,cosine_added_ind_nom_pens_ult1,jacob_added_ind_recibo_ult1,cosine_added_ind_recibo_ult1
ncodpers,,,,,,,,,,,,,,,,,,,,,
15892,0.0,0.0,0.0,0.0,0.036062,0.082235,0.000141,0.00449,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.190078,0.357068,0.194755,0.364195,0.000000,0.000000
15897,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000141,0.00449,0.000000,0.000000,...,0.000000,0.000000,0.0,0.0,0.193767,0.370162,0.198642,0.377699,0.000000,0.000000
15906,0.0,0.0,0.0,0.0,0.025829,0.064539,0.000141,0.00449,0.000000,0.000000,...,0.005036,0.019046,0.0,0.0,0.185333,0.347796,0.190196,0.355342,0.000000,0.000000
15925,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.009018,0.019958,...,0.002970,0.011834,0.0,0.0,0.013948,0.030210,0.014455,0.031224,0.023277,0.055703
15927,0.0,0.0,0.0,0.0,0.011580,0.032723,0.000141,0.00449,0.016002,0.047308,...,0.000824,0.005028,0.0,0.0,0.018451,0.044834,0.018600,0.045200,0.000000,0.000000


time: 40.1 ms


In [24]:
simi_feat, ind = get_similarity_features(('data/train_2015_05_28.csv', 'data/added_product_2015_05_28.csv', 1))

data/added_product_2015_05_28.csv


100%|██████████| 41745/41745 [03:14<00:00, 215.02it/s]

time: 3min 18s


In [25]:
simi_feat.head()

,jacob_ind_ahor_fin_ult1,cosine_ind_ahor_fin_ult1,jacob_ind_aval_fin_ult1,cosine_ind_aval_fin_ult1,jacob_ind_cco_fin_ult1,cosine_ind_cco_fin_ult1,jacob_ind_cder_fin_ult1,cosine_ind_cder_fin_ult1,jacob_ind_cno_fin_ult1,cosine_ind_cno_fin_ult1,...,jacob_ind_valo_fin_ult1,cosine_ind_valo_fin_ult1,jacob_ind_viv_fin_ult1,cosine_ind_viv_fin_ult1,jacob_ind_nomina_ult1,cosine_ind_nomina_ult1,jacob_ind_nom_pens_ult1,cosine_ind_nom_pens_ult1,jacob_ind_recibo_ult1,cosine_ind_recibo_ult1
ncodpers,,,,,,,,,,,,,,,,,,,,,
15892,0.001922,0.042186,0.001856,0.084516,0.315905,1.033608,0.010739,0.118331,0.000000,0.000000,...,0.000000,0.000000,0.079300,0.317117,1.662739,2.576130,1.702099,2.618801,0.000000,0.000000
15897,0.003068,0.068065,0.002182,0.096053,0.000000,0.000000,0.017038,0.181267,0.000000,0.000000,...,0.000000,0.000000,0.114651,0.481117,1.750465,2.822744,1.789703,2.866702,0.000000,0.000000
15906,0.001816,0.041220,0.001136,0.056974,0.362383,1.063298,0.006653,0.085441,0.000000,0.000000,...,0.439234,0.959849,0.074186,0.322716,1.462919,2.324312,1.492736,2.353052,0.000000,0.000000
15925,0.000642,0.024398,0.000412,0.020984,0.000000,0.000000,0.002701,0.050439,0.344700,0.603161,...,0.171709,0.454029,0.030876,0.177510,0.308773,0.546272,0.313498,0.552792,0.425316,0.840093
15927,0.001608,0.023118,0.000558,0.025156,0.150713,0.511194,0.006105,0.046694,0.635437,1.164962,...,0.224904,0.506582,0.045052,0.153827,0.538950,1.053044,0.546324,1.061477,0.000000,0.000000


time: 41.1 ms


In [26]:
def get_wts_from_mat(df, row):
    brow = map(bool, row)
    wts = df.loc[brow, :].apply(np.mean)
    wts[brow] = 0
    return wts

time: 2.74 ms


In [27]:
simi_feat1, ind = get_similarity_features(('data/train_2015_05_28.csv', 'data/added_product_2015_05_28.csv', 1))

data/added_product_2015_05_28.csv


100%|██████████| 41745/41745 [05:38<00:00, 123.49it/s]

time: 5min 41s


In [28]:
simi_feat1.head()

,jacob_ind_ahor_fin_ult1,cosine_ind_ahor_fin_ult1,jacob_ind_aval_fin_ult1,cosine_ind_aval_fin_ult1,jacob_ind_cco_fin_ult1,cosine_ind_cco_fin_ult1,jacob_ind_cder_fin_ult1,cosine_ind_cder_fin_ult1,jacob_ind_cno_fin_ult1,cosine_ind_cno_fin_ult1,...,jacob_ind_valo_fin_ult1,cosine_ind_valo_fin_ult1,jacob_ind_viv_fin_ult1,cosine_ind_viv_fin_ult1,jacob_ind_nomina_ult1,cosine_ind_nomina_ult1,jacob_ind_nom_pens_ult1,cosine_ind_nom_pens_ult1,jacob_ind_recibo_ult1,cosine_ind_recibo_ult1
ncodpers,,,,,,,,,,,,,,,,,,,,,
15892,0.000275,0.006027,0.000265,0.012074,0.045129,0.147658,0.001534,0.016904,0.000000,0.000000,...,0.000000,0.000000,0.011329,0.045302,0.237534,0.368019,0.243157,0.374114,0.000000,0.000000
15897,0.000307,0.006807,0.000218,0.009605,0.000000,0.000000,0.001704,0.018127,0.000000,0.000000,...,0.000000,0.000000,0.011465,0.048112,0.175046,0.282274,0.178970,0.286670,0.000000,0.000000
15906,0.000259,0.005889,0.000162,0.008139,0.051769,0.151900,0.000950,0.012206,0.000000,0.000000,...,0.062748,0.137121,0.010598,0.046102,0.208988,0.332045,0.213248,0.336150,0.000000,0.000000
15925,0.000214,0.008133,0.000137,0.006995,0.000000,0.000000,0.000900,0.016813,0.114900,0.201054,...,0.057236,0.151343,0.010292,0.059170,0.102924,0.182091,0.104499,0.184264,0.141772,0.280031
15927,0.000322,0.004624,0.000112,0.005031,0.030143,0.102239,0.001221,0.009339,0.127087,0.232992,...,0.044981,0.101316,0.009010,0.030765,0.107790,0.210609,0.109265,0.212295,0.000000,0.000000


time: 47.2 ms


---
# Custom Jacobian similarities
---

In [7]:
tr = pd.read_csv('data/train_2015_05_28.csv', header=None, names=[HEADER[1]]+HEADER[23:], usecols=[1]+range(23,48))

time: 2.35 s


In [8]:
tr.columns

Index([u'ncodpers', u'segmento', u'ind_ahor_fin_ult1', u'ind_aval_fin_ult1',
       u'ind_cco_fin_ult1', u'ind_cder_fin_ult1', u'ind_cno_fin_ult1',
       u'ind_ctju_fin_ult1', u'ind_ctma_fin_ult1', u'ind_ctop_fin_ult1',
       u'ind_ctpp_fin_ult1', u'ind_deco_fin_ult1', u'ind_deme_fin_ult1',
       u'ind_dela_fin_ult1', u'ind_ecue_fin_ult1', u'ind_fond_fin_ult1',
       u'ind_hip_fin_ult1', u'ind_plan_fin_ult1', u'ind_pres_fin_ult1',
       u'ind_reca_fin_ult1', u'ind_tjcr_fin_ult1', u'ind_valo_fin_ult1',
       u'ind_viv_fin_ult1', u'ind_nomina_ult1', u'ind_nom_pens_ult1',
       u'ind_recibo_ult1'],
      dtype='object')

time: 3.16 ms


In [14]:
parti = tr.loc[tr.segmento == '02 - PARTICULARES', :]
top = tr.loc[tr.segmento == '01 - TOP', :]
uni = tr.loc[tr.segmento == '03 - UNIVERSITARIO', :]

time: 278 ms


In [11]:
parti.shape

(368666, 26)

time: 3.05 ms


In [42]:
parti = tr.loc[tr.segmento == '02 - PARTICULARES', :]
top = tr.loc[tr.segmento == '01 - TOP', :]
uni = tr.loc[tr.segmento == '03 - UNIVERSITARIO', :]

p_o_f_parti = get_persons_of_feature(parti)
p_o_f_top = get_persons_of_feature(top)
p_o_f_uni = get_persons_of_feature(uni)

time: 842 ms


In [46]:
parti.segmento.value_counts()

02 - PARTICULARES    372285
Name: segmento, dtype: int64

time: 27.5 ms


In [15]:
tr.segmento.fillna('02 - PARTICULARES', inplace=True)

time: 32.4 ms


In [43]:
jacob_pd_p, cosine_pd_p = get_similarity_matrices(p_o_f_parti)
jacob_pd_t, cosine_pd_t = get_similarity_matrices(p_o_f_top)
jacob_pd_u, cosine_pd_u = get_similarity_matrices(p_o_f_uni)

AttributeError: 'dict' object has no attribute 'loc'

time: 355 ms


In [44]:
get_similarity_matrices(tr)

(          0         1         2         3         4         5         6   \
 0   1.000000  0.000000  0.000104  0.000000  0.000142  0.000000  0.000000   
 1   0.000000  1.000000  0.000020  0.000000  0.000237  0.000000  0.000141   
 2   0.000104  0.000020  1.000000  0.000465  0.011995  0.000002  0.007777   
 3   0.000000  0.000000  0.000465  1.000000  0.000990  0.000000  0.000000   
 4   0.000142  0.000237  0.011995  0.000990  1.000000  0.000000  0.014946   
 5   0.000000  0.000000  0.000002  0.000000  0.000000  1.000000  0.000000   
 6   0.000000  0.000141  0.007777  0.000000  0.014946  0.000000  1.000000   
 7   0.000371  0.000057  0.131785  0.001046  0.065815  0.000000  0.000411   
 8   0.000309  0.000225  0.037456  0.001090  0.135114  0.000000  0.000705   
 9   0.000000  0.000000  0.002544  0.000419  0.001698  0.000000  0.010690   
 10  0.000000  0.000000  0.002377  0.000559  0.004226  0.000000  0.001168   
 11  0.000199  0.000085  0.046653  0.000819  0.084756  0.000585  0.010875   

time: 1.15 s
